### General Imports

In [ ]:
import json
import os
from argparse import ArgumentParser
import pandas as pd

### Data Creation

In [ ]:
source1 = """PatientID,FirstName,LastName,Gender,Age,DateOfBirth,Phone,Email
101,John,Smith,Male,35,15-07-1987,(555) 123-4567,john.smith@email.com
102,Mary,Johnson,Female,28,22-03-1995,(555) 987-6543,mary.j@email.com
103,David,Williams,Male,45,10-12-1978,(555) 555-5555,david.w@email.com
104,Sarah,Brown,Female,52,05-09-1971,(555) 111-2222,sarah.b@email.com
105,Michael,Davis,Male,30,20-11-1992,(555) 333-4444,michael.d@email.com
"""

source2="""PatientID,Name,Sex,Age,DOB,Telephone,Email
101,John Smith,M,35,15/07/1987,(555) 123-4567,john.smith@email.com
102,Mary Johnson,F,28,22/03/1995,(555) 987-6543,mary.j@email.com
103,David Williams,M,45,10/12/1978,(555) 555-5555,david.w@email.com
104,Sarah Brown,F,52,05/09/1971,(555) 111-2222,sarah.b@email.com
105,Michael Davis,M,30,20/11/1992,(555) 333-4444,michael.d@email.com
"""

sample="""ID,Full Name,Gender,Age,DOB,Mobile,Email
104,Sarah Brown,Female,52,1971-09-05,555-111-2222,sarah.b@email.com
105,Michael Davis,Male,30,1992-11-20,555-333-4444,michael.d@email.com
"""

with open("source1.csv", "w") as fp:
  fp.write(source1)

with open("source2.csv", "w") as fp:
  fp.write(source2)

with open("sample.csv", "w") as fp:
  fp.write(sample)

source1_df = pd.read_csv("source1.csv")
source2_df = pd.read_csv("source2.csv")
sample_df = pd.read_csv("sample.csv")

In [ ]:
source1_df

,PatientID,FirstName,LastName,Gender,Age,DateOfBirth,Phone,Email
0,101,John,Smith,Male,35,15-07-1987,(555) 123-4567,john.smith@email.com
1,102,Mary,Johnson,Female,28,22-03-1995,(555) 987-6543,mary.j@email.com
2,103,David,Williams,Male,45,10-12-1978,(555) 555-5555,david.w@email.com
3,104,Sarah,Brown,Female,52,05-09-1971,(555) 111-2222,sarah.b@email.com
4,105,Michael,Davis,Male,30,20-11-1992,(555) 333-4444,michael.d@email.com


In [ ]:
source2_df

,PatientID,Name,Sex,Age,DOB,Telephone,Email
0,101,John Smith,M,35,15/07/1987,(555) 123-4567,john.smith@email.com
1,102,Mary Johnson,F,28,22/03/1995,(555) 987-6543,mary.j@email.com
2,103,David Williams,M,45,10/12/1978,(555) 555-5555,david.w@email.com
3,104,Sarah Brown,F,52,05/09/1971,(555) 111-2222,sarah.b@email.com
4,105,Michael Davis,M,30,20/11/1992,(555) 333-4444,michael.d@email.com


In [ ]:
sample_df

,ID,Full Name,Gender,Age,DOB,Mobile,Email
0,104,Sarah Brown,Female,52,1971-09-05,555-111-2222,sarah.b@email.com
1,105,Michael Davis,Male,30,1992-11-20,555-333-4444,michael.d@email.com


### Utility Methods around Mistral 7B Instruct Model

In [ ]:
#!pip install optimum
#!pip install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
#!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
import re

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"




# return model and tokenizer
def get_model():
  model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

  tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
  return model, tokenizer

# get prediction
def get_prediction(model, tokenizer, prompt_template):
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=1000)
  return tokenizer.decode(output[0])

# create prompt template rowwise and calling the model
def predict(model, tokenizer, source1_df=source1_df, source2_df=source2_df, sample_df=sample_df):
  source1_row = source1_df.iloc[:2].to_json()
  source2_row = source2_df.iloc[:2].to_json()
  sample_row = sample_df.iloc[:2].to_json()

  prompt = template = f"""You are an assistant to generate code.

  Lets think step by step

  1. You are given three tables. Source1, Source2 and Sample.
  2. Task is to generate a target table which has exactly the same number of columns as sample table and same number of rows as source1 table
  3. For each column in the sample table, identify which column matches from source1 or source2 table and find the transformation needed from source to sample table
  4. Use pandas in built functions or regex and transform the column into sample table format.
  5. Apply mobile transformations(xxx-xxx-xxxx) simillar to the sample table format.
  6. Always transform dates into yyyy-mm-dd format
  7. Do not change the source1, source2 and sample table values. Instead, find the transformations and apply it on the target table.
  8. Do not perform merge or concat, as the tables are huge.
  9. The column names in the sample table might not match exactly in the source1 and source2 table. identify the columns based on the column values.
  10. Generate python code to create target table by reading source1.csv, source2.csv, sample.csv.

  Few rows of Source1, Source2 and Sample tables:

  Source 1 - {source1_row}
  Source 2 - {source2_row}
  Sample - {sample_row}

  Python Code:
  """
  prompt_template=f'''<s>[INST] {prompt} [/INST]
    '''
  predicted_text = get_prediction(model, tokenizer, prompt_template)
  return predicted_text[predicted_text.index("[/INST]")+len("[/INST]"):]


# Executing - getting model and tokenizer
model, tokenizer = get_model()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Executing - Prediction
predicted_text = predict(model, tokenizer)
print(predicted_text)


    
    ```python
import pandas as pd
import re

# Read source1, source2 and sample tables
source1 = pd.read_csv('source1.csv')
source2 = pd.read_csv('source2.csv')
sample = pd.read_csv('sample.csv')

# Create a new target table with the same number of columns as sample table and the same number of rows as source1 table
target = pd.DataFrame(columns=sample.columns)
target.index = source1.index

# Iterate through each column in the sample table
for col_name, col_value in sample.columns.items():
    # Identify which column in source1 or source2 table matches the column in sample table
    if col_name == 'ID':
        target[col_name] = source1[col_name]
    elif col_name == 'Full Name':
        target[col_name] = sample[col_name]
    elif col_name == 'Gender':
        target[col_name] = sample[col_name]
    elif col_name == 'Age':
        target[col_name] = sample[col_name]
    elif col_name == 'DOB':
        target[col_name] = sample[col_name]
    elif col_name == 'Mobile':
        ta